In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_csv('data.csv', header= None)
label_df = pd.read_csv('label.csv', header= None)

In [3]:
data_df = pd.DataFrame(data_df)
label_df = pd.DataFrame(label_df)

In [4]:
data_df = np.array(data_df)
label_df = np.array(label_df)

In [1]:

import math

def _popcnt(v):
	c = 0
	while v:
		v &= v - 1
		c += 1
	return c
def _bit_length(x):
	return len(bin(x))-2
try:
	int(0).bit_length()
	_bit_length = lambda x: x.bit_length()
except:
	pass

def ham_dist(x,y):
	return _popcnt(x^y)
def ham_centers_factory(data):

	l = max((_bit_length(d) for d in data))
	buckets = [0]*l
	bits = map(lambda i: 1<<i,range(l))
	for d in data:
		for i,two_to_the_i in enumerate(bits):
			if d&two_to_the_i:
				buckets[i] += 1
			else:
				buckets[i] -= 1
	return reduce(
		lambda accum,x: accum|(x[1]>0)*bits[x[0]],
		enumerate(buckets),
		0
	)

def euc_dist(x,y):
	return sum((math.pow(x[i]-y[i],2) for i in range(len(x))))
def euc_centers_factory(data):
	dim = len(data[0])
	n = len(data)
	sums = [sum((data[j][i] for j in range(n))) for i in range(dim)]
	return [float(s)/n for s in sums]

def kmeans(data,centers,distf,centersdf,cutoff):

	k = len(centers)
	while True:
		distances = [map(lambda x: distf(x,y),centers) for y in data]
		data_to_centroids = [min(enumerate(x),key=lambda x:x[1])[0] for x in distances]
		
		new_centroids = map(
			centersdf, 
			[
				map(
					lambda x: data[x[0]], 
					filter(
						lambda x: x[1]==y,
						enumerate(data_to_centroids)
					) 
				)
				for y in range(k)
			]
		)
		
		changes = [distf(new_centroids[i],centers[i]) for i in range(k)]
		
		if max(changes)<=cutoff:
			return centers,data_to_centroids,distances
		
		centers = new_centroids

def kmeans_euc(data,centers,cutoff):
	return kmeans(data,centers,euc_dist,euc_centers_factory,cutoff)

def kmeans_ham(data,centers,cutoff):
	return kmeans(data,centers,ham_dist,ham_centers_factory,cutoff)

In [6]:
kmeans_euc(data_df, label_df, 0.1)
kmeans_ham(data_df, label_df, 0.1)

TypeError: 'map' object is not subscriptable